In [1]:
!pip install patchify

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as implt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()


from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from PIL import Image
from tensorflow.keras.optimizers import Adam
import random
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from patchify import patchify
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.saving import load_model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# !rm -r /kaggle/working

rm: cannot remove '/kaggle/working': Device or resource busy


In [2]:
import shutil

# Specify the path to the folder you want to zip
folder_path = '/kaggle/working/files'

# Specify the name for the zip file (including the .zip extension)
zip_file_name = '/kaggle/working/files.zip'

# Create a zip archive of the folder
shutil.make_archive(zip_file_name.split('.zip')[0], 'zip', folder_path)

'/kaggle/working/files.zip'

In [3]:
root_dir = "/kaggle/input/gtsrb-climatic"
subdirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
classes = {i: subdir for i, subdir in enumerate(subdirs)}

In [4]:
hp = {}
hp["image_size"] = 81
hp["num_channels"] = 3
hp["patch_size"] = 3
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 8
hp["lr"] = 1e-3
hp["num_epochs"] = 128
hp["num_classes"] = 43
hp["class_names"] = [str(i) for i in list(classes.values())]

hp["hidden_dim"] = 3*3*3
hp["mlp_dim"] = 3072
hp["num_heads"] = 3
hp["dropout_rate"] = 0.1
hp["num_layers"] = 12

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def load_data(path):
    images = shuffle(glob(os.path.join(path, "*", "*.jpg")))
    return images

In [6]:
os.listdir('/kaggle/working/')

['files.zip', 'files', 'PreGAN_ViT_30Epochs.h5', '.virtual_documents']

In [7]:
def process_image_label(path):
    """ Reading images """
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
    image = image/255.0

    """ Preprocessing to patches """
    patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
    patches = patchify(image, patch_shape, hp["patch_size"])

    patches = np.reshape(patches, hp["flat_patches_shape"])
    patches = patches.astype(np.float32)

    """ Label """
    class_name = path.split("/")[-2]
    class_idx = hp["class_names"].index(class_name)
    class_idx = np.array(class_idx, dtype=np.int32)

    return patches, class_idx

def parse(path):
    patches, labels = tf.numpy_function(process_image_label, [path], [tf.float32, tf.int32])
    labels = tf.one_hot(labels, hp["num_classes"])

    patches.set_shape(hp["flat_patches_shape"])
    labels.set_shape(hp["num_classes"])

    return patches, labels

def tf_dataset(images, batch=16):
    ds = tf.data.Dataset.from_tensor_slices((images))
    ds = ds.map(parse).batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

In [8]:
""" Directory for storing files """
create_dir("files")

""" Paths """
dataset_path = "/kaggle/working/"
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "log.csv")

""" Dataset """
train_x = load_data('/kaggle/input/pjt-gtsrb/GTSRB')
valid_x = load_data('/kaggle/input/gtsrb-climatic')
print(f"Train: {len(train_x)} - Valid: {len(valid_x)}")

train_ds = tf_dataset(train_x, batch=hp["batch_size"])
valid_ds = tf_dataset(valid_x, batch=hp["batch_size"])

Train: 39209 - Valid: 39209


In [9]:
from tensorflow.keras import layers as L
from tensorflow.keras.models import Model

In [10]:
class ClassToken(L.Layer):
    def __init__(self):
        super().__init__()
    
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )
        
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]
        
        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls  = tf.cast(cls, dtype=inputs.dtype)
        
        return cls

In [11]:
def MLP(x, cf):
    x = L.Dense(cf["mlp_dim"], activation="gelu")(x)
    x = L.Dropout(cf["dropout_rate"])(x)
    x = L.Dense(cf["hidden_dim"])(x)
    x = L.Dropout(cf["dropout_rate"])(x)
    
    return x

In [12]:
def transformer_encoder(x, cf):
    skip_1 = x
    x = L.LayerNormalization()(x)
    x = L.MultiHeadAttention(
        num_heads=cf["num_heads"],
        key_dim=cf["hidden_dim"]
    )(x, x)
    x = L.Add()([x, skip_1])
    
    skip_2 = x
    x = L.LayerNormalization()(x)
    x = MLP(x, cf)
    x = L.Add()([x, skip_2])
    
    return x

In [13]:
def ViT(cf):
    """ Input """
    input_shape = (cf["num_patches"], 
                   cf["patch_size"]*cf["patch_size"]*cf["num_channels"])
    inputs = L.Input(shape=input_shape)                                        # (None, 256, 768)
    
    """ Patch + Position Embeddings """
    patch_embed = L.Dense(cf["hidden_dim"])(inputs)                            # (None, 256, 768)
    
    positions = tf.range(start=0, limit=cf["num_patches"], delta=1)
    pos_embed = L.Embedding(input_dim=cf["num_patches"], 
                            output_dim=cf["hidden_dim"])(positions)            # (256, 768)
    
    embed = patch_embed + pos_embed                                            # (None, 256, 768)
    
    """ Adding Class Token"""
    token = ClassToken()(embed)
    x = L.Concatenate(axis=1)([token, embed])                                  # (None, 257, 768)
    
    for _ in range(cf["num_layers"]):
        x = transformer_encoder(x, cf)    
    
    """ Classification Head """
    x = L.LayerNormalization()(x)                                              # (None, 257, 768)
    x = x[:, 0, :]                                                             # (None, 768)
    outputs = L.Dense(cf["num_classes"], activation="softmax")(x)              # (None, 4)
    
    model = Model(inputs, outputs, name="Vision-Transformer")
    return model

In [14]:
# Create custom F1 Score Metrics

from keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [ ]:
# with strategy.scope():
#     model = ViT(hp)
#     model.compile(loss="binary_crossentropy",
#             optimizer=tf.keras.optimizers.Adam(learning_rate=hp["lr"], clipvalue=1.0),
#             metrics=["accuracy",
#                 tf.keras.metrics.Precision(name="precision"),
#                 f1_score,
#                 tf.keras.metrics.Recall(name="recall"),
#                 tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy"),
#                 tf.keras.metrics.AUC(name="auc")])

In [17]:
# Define the custom objects dictionary
custom_objects = {'f1_score': f1_score,
                 'ClassToken': ClassToken}

with strategy.scope():
    model = load_model('/kaggle/working/PreGAN_ViT_30Epochs.h5', custom_objects=custom_objects)

In [18]:
print(model.input_shape, model.output_shape)

(None, 729, 27) (None, 43)


In [19]:
# Vision Transformer Callbacks
callbacks = [
        ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-8, verbose=1),
        CSVLogger(csv_path),
    ]

In [20]:
ViT_History = model.fit(train_ds,
                      epochs = hp["num_epochs"],
                      batch_size = hp["batch_size"],
                      steps_per_epoch = len(train_ds),
                      validation_data = valid_ds,
                      validation_steps = len(valid_ds),
                      callbacks=callbacks)

Epoch 1/128
4902/4902 [==============================] - ETA: 0s - loss: 0.0011 - accuracy: 0.9940 - precision: 0.9942 - f1_score: 0.9938 - recall: 0.9937 - categorical_accuracy: 0.9940 - auc: 0.9996
Epoch 1: val_loss improved from inf to 0.10512, saving model to files/model.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4902/4902 [==============================] - 1366s 266ms/step - loss: 0.0011 - accuracy: 0.9940 - precision: 0.9942 - f1_score: 0.9938 - recall: 0.9937 - categorical_accuracy: 0.9940 - auc: 0.9996 - val_loss: 0.1051 - val_accuracy: 0.5971 - val_precision: 0.6080 - val_f1_score: 0.5996 - val_recall: 0.5938 - val_categorical_accuracy: 0.5971 - val_auc: 0.8378 - lr: 1.0000e-04
Epoch 2/128
4902/4902 [==============================] - ETA: 0s - loss: 8.9107e-04 - accuracy: 0.9949 - precision: 0.9951 - f1_score: 0.9948 - recall: 0.9947 - categorical_accuracy: 0.9949 - auc: 0.9997
Epoch 2: val_loss did not improve from 0.10512
4902/4902 [==============================] - 1290s 263ms/step - loss: 8.9107e-04 - accuracy: 0.9949 - precision: 0.9951 - f1_score: 0.9948 - recall: 0.9947 - categorical_accuracy: 0.9949 - auc: 0.9997 - val_loss: 0.1086 - val_accuracy: 0.5917 - val_precision: 0.6033 - val_f1_score: 0.5944 - val_recall: 0.5883 - val_categorical_accuracy: 0.5917 - val_auc: 0.8331 - lr: 1.

KeyboardInterrupt: 

In [21]:
model.save('/kaggle/working/PreGAN_ViT_46Epochs.h5')

In [ ]:
#######################################


COPY HISTORYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY


#######################################

In [ ]:
history_df = pd.DataFrame(ViT_History.history)
history_df.to_csv("/kaggle/working/files/history_df.csv")

In [ ]:
# !rm -r /kaggle/working